In [1]:
from experiment.scenario import *
from experiment_setup.exp_utrecht_10_492_594_v2_setup import *

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Setup

Set up experiment:

In [2]:
load_existing_experiment = True

if not load_existing_experiment:
    e = Experiment(experiment_name, experiment_dirpath, default_config, scenario_settings, scene_parts)

    print("\nSetting up experiment:")
    e.setup()
else:
    print("\nLoading existing experiment:")
    e = Experiment.load(experiment_dirpath / experiment_name, load_scenarios=True)


Loading existing experiment:

Loading experiment configuration ...
Initializing experiment ...
Loading scenarios ...
- 143 scenarios


# Try reconstruction with different parameter sets

In [3]:
scenario_id = 0
e[scenario_id].setup_reconstruction_optimization()
e[scenario_id].recon_optim.load_optim_experiment()


Setting up reconstruction optimization ...
- Setting up experiment ...
- Setting up optimizer ...

Loading experiment configuration ...
File `scenario_settings.json` not found.
Initializing experiment ...
Loading scenarios ...
- 51 scenarios


In [4]:
optim_scenario_id = 0
params = e[scenario_id].recon_optim.optim_experiment[optim_scenario_id].geoflow_parameters
print(json.dumps(params, indent=2, ensure_ascii=False))

{
  "r_plane_epsilon": 0.2,
  "r_plane_k": 15,
  "r_plane_min_points": 15,
  "r_plane_normal_angle": 0.75,
  "thres_alpha": 0.25,
  "r_line_epsilon": 0.4,
  "thres_reg_line_dist": 0.5,
  "thres_reg_line_ext": 1.5,
  "r_optimisation_data_term": 7.0,
  "r_normal_k": 5
}


In [ ]:
e[scenario_id].set_reconstruction_params(params)

In [ ]:
e[scenario_id].setup_reconstruction()
e[scenario_id].prepare_reconstruction()
e[scenario_id].run_reconstruction()

# Select best optimization scenario

In [ ]:
scenario_id = 49

In [ ]:
e[scenario_id].setup_reconstruction_optimization()

In [ ]:
e[scenario_id].recon_optim.load_optim_experiment()

In [ ]:
e[scenario_id].recon_optim.target_metric

In [ ]:
e[scenario_id].recon_optim.select_optimal_scenario()

# Rerun individual optimization scenarios

Mostly in cases where the Geoflow output is damaged.

In [ ]:
optim_id = 31
e[scenario_id].recon_optim.optim_experiment[optim_id].setup_reconstruction()
e[scenario_id].recon_optim.optim_experiment[optim_id].prepare_reconstruction()
e[scenario_id].recon_optim.optim_experiment[optim_id].run_reconstruction(add_geoflow_params=e[scenario_id].recon_optim.add_geoflow_params)

In [ ]:
e[scenario_id].recon_optim.optim_experiment[optim_id].setup_evaluation(lods=["2.2"])
e[scenario_id].recon_optim.optim_experiment[optim_id].run_evaluation(evaluator_selection="hausdorff")

# Full indpendent optimization + renaming

Perform a full independent optimization experiment for a point cloud scenario even though the optimization experiment with this name exists already by renaming the new optimization experiment before execution.

In [3]:
scenario_id = 80

In [4]:
e[scenario_id].setup_reconstruction_optimization(recon_timeout=600)


Setting up reconstruction optimization ...
- Setting up experiment ...
- Setting up optimizer ...


Create the required experiment folders so that it can be renamed. Then, rename it and change the corresponding variables of the `ReconstructionOptimization` instance.

In [5]:
e[scenario_id].recon_optim.optim_experiment.dirpath.mkdir()
e[scenario_id].recon_optim.optim_experiment.settings_dirpath.mkdir()

In [6]:
e[scenario_id].recon_optim.optim_experiment.rename("scenario_080_full_optim")

Renaming experiment from `scenario_080` to `scenario_080_full_optim` ...
Saving experiment configuration ...
Saving scenario configurations ...


WindowsPath('C:/Users/Florian/Data/city-to-scan-to-city/utrecht_10-492-594_v2/06_reconstruction_optimization/scenario_080_full_optim')

In [7]:
e[scenario_id].recon_optim.output_dirpath = e[scenario_id].recon_optim.optim_experiment.dirpath

Now, the `prepare()` method can be called, which sets up the logger in the correct folder with the new name. Add the default parameter set for probing and run the optimization.

In [8]:
e[scenario_id].recon_optim.prepare()


Preparing reconstruction optimization ...
Saving experiment configuration ...
Saving scenario configurations ...
- Setting up logger ...


In [9]:
geoflow_default_params_for_optim = {
    k: glb.geoflow_parameters_default[k]
    for k in [k2.split("_", 1)[1] for k2 in glb.geoflow_optim_parameter_space_narrow_2.keys()]
}

In [10]:
e[scenario_id].recon_optim.optimizer.probe(
    params=geoflow_default_params_for_optim,
    lazy=True
)

In [11]:
e[scenario_id].run_reconstruction_optimization(init_points=40, n_iter=160)


Running reconstruction optimization ...

Starting optimization scenario 'optim_0000' with the following settings:
{
  "r_line_epsilon": 0.4,
  "r_optimisation_data_term": 7.0,
  "r_plane_epsilon": 0.2,
  "r_plane_min_points": 15,
  "thres_alpha": 0.25,
  "thres_reg_line_dist": 0.5,
  "thres_reg_line_ext": 1.5,
  "r_plane_k": 15,
  "scenario_name": "optim_0000"
}

Starting 3D building reconstruction ...
- Command: geof C:/Users/Florian/Data/city-to-scan-to-city/utrecht_10-492-594_v2/06_reconstruction_optimization/scenario_080_full_optim/07_reconstruction/optim_0000/reconstruct.json --verbose --workdir --config C:/Users/Florian/Data/city-to-scan-to-city/utrecht_10-492-594_v2/06_reconstruction_optimization/scenario_080_full_optim/07_reconstruction/optim_0000/config.toml
- Output log file: C:\Users\Florian\Data\city-to-scan-to-city\utrecht_10-492-594_v2\06_reconstruction_optimization\scenario_080_full_optim\07_reconstruction\optim_0000\geoflow_log_241012-110957.txt

Finished 3D building r

# Rerun optim_0000 (default params) for high-density scenarios

For the last few high-density scenarios, the optimization iteration with Geoflow default parameter values ran into the timeout. But since these results are particularly interesting, it could be cool to run them again without timeout.

In [44]:
scenario_ids = list(range(116, 121))

for scenario_id in scenario_ids:
    e[scenario_id].setup_reconstruction_optimization()
    e[scenario_id].recon_optim.load_optim_experiment()
    
    add_geoflow_params = {"skip_lod12": True, "skip_lod13": True}
    e[scenario_id].recon_optim.optim_experiment[0].setup_reconstruction(add_geoflow_params=add_geoflow_params, geoflow_timeout=None)
    e[scenario_id].recon_optim.optim_experiment[0].prepare_reconstruction()
    
    e[scenario_id].recon_optim.optim_experiment[0].run_reconstruction()

    e[scenario_id].recon_optim.optim_experiment[0].setup_evaluation(lods="2.2")
    e[scenario_id].recon_optim.optim_experiment[0].run_evaluation(evaluator_selection=["hausdorff", "complexity", "complexity_diff", "geoflow_output"])
    
    e[scenario_id].recon_optim.optim_experiment[0].concat_evaluation_results(evaluator_selection=["hausdorff", "complexity", "complexity_diff", "geoflow_output"])
    
    e[scenario_id].recon_optim.optim_experiment.run_steps(Scenario.setup_evaluation, lods=["2.2"])
    e[scenario_id].recon_optim.optim_experiment.compute_summary_statistics(evaluator_selection=["hausdorff", "complexity", "complexity_diff", "geoflow_output"], ignore_missing=True)


Setting up reconstruction optimization ...
- Setting up experiment ...
- Setting up optimizer ...

Loading experiment configuration ...
File `scenario_settings.json` not found.
Initializing experiment ...
Loading scenarios ...
- 51 scenarios

Starting 3D building reconstruction ...
- Command: geof C:/Users/Florian/Data/city-to-scan-to-city/utrecht_10-492-594_v2/06_reconstruction_optimization/scenario_116/07_reconstruction/optim_0000/reconstruct.json --verbose --workdir --config C:/Users/Florian/Data/city-to-scan-to-city/utrecht_10-492-594_v2/06_reconstruction_optimization/scenario_116/07_reconstruction/optim_0000/config.toml
- Output log file: C:\Users\Florian\Data\city-to-scan-to-city\utrecht_10-492-594_v2\06_reconstruction_optimization\scenario_116\07_reconstruction\optim_0000\geoflow_log_241014-191140.txt

Finished 3D building reconstruction after 0:16:22.970975.

Starting GeopackageBuildingsEvaluator ...

Finished GeopackageBuildingsEvaluator after 0:00:00.321121.

Reconstruction 

# Rerun optim_0000 for scenario_048 and 037

In [31]:
scenario_id = 37

In [32]:
e[scenario_id].setup_reconstruction_optimization()
e[scenario_id].recon_optim.load_optim_experiment()


Setting up reconstruction optimization ...
- Setting up experiment ...
- Setting up optimizer ...

Loading experiment configuration ...
File `scenario_settings.json` not found.
Initializing experiment ...
Loading scenarios ...
- 51 scenarios


In [33]:
add_geoflow_params = {"skip_lod12": True, "skip_lod13": True}
e[scenario_id].recon_optim.optim_experiment[0].setup_reconstruction(add_geoflow_params=add_geoflow_params, geoflow_timeout=None)
e[scenario_id].recon_optim.optim_experiment[0].prepare_reconstruction()

In [34]:
e[scenario_id].recon_optim.optim_experiment[0].run_reconstruction()


Starting 3D building reconstruction ...
- Command: geof C:/Users/Florian/Data/city-to-scan-to-city/utrecht_10-492-594_v2/06_reconstruction_optimization/scenario_037/07_reconstruction/optim_0000/reconstruct.json --verbose --workdir --config C:/Users/Florian/Data/city-to-scan-to-city/utrecht_10-492-594_v2/06_reconstruction_optimization/scenario_037/07_reconstruction/optim_0000/config.toml
- Output log file: C:\Users\Florian\Data\city-to-scan-to-city\utrecht_10-492-594_v2\06_reconstruction_optimization\scenario_037\07_reconstruction\optim_0000\geoflow_log_241014-184401.txt

Finished 3D building reconstruction after 0:00:08.692701.

Starting GeopackageBuildingsEvaluator ...

Finished GeopackageBuildingsEvaluator after 0:00:00.044554.

Reconstruction in optim_0000 yielded 99 buildings.


In [35]:
e[scenario_id].recon_optim.optim_experiment[0].setup_evaluation(lods="2.2")

In [36]:
e[scenario_id].recon_optim.optim_experiment[0].run_evaluation(evaluator_selection=["hausdorff", "complexity", "complexity_diff", "geoflow_output"])


Starting HausdorffLODSEvaluator ...

This HausdorffLODSEvaluator is bidirectional.

Setting up a HausdorffEvaluator (bidirectional) for each LOD ...

Running HausdorffEvaluator for LOD '22' ...

Splitting OBJ files into one file per individual object ...

Identifying individual OBJ files present in both input datasets ...
- Directories:
  Input 1: C:\Users\Florian\Data\city-to-scan-to-city\utrecht_10-492-594_v2\01_input\OBJ\10-492-594_LOD22_mat_python
  Input 2: C:\Users\Florian\Data\city-to-scan-to-city\utrecht_10-492-594_v2\06_reconstruction_optimization\scenario_037\07_reconstruction\optim_0000\output\model_lod22
- File comparison: Found 1084 unique OBJ file names in total, of which
  99 file names present both directories
  985 file names present only in directory 1
  0 file names present only in directory 2
Computing bidirectional Hausdorff distances between split OBJ files ...

- All done.

Joining results ...

Finished HausdorffLODSEvaluator after 0:00:05.538866.

Starting Comp

Summary stats from first attempt (after original):

In [8]:
e[48].recon_optim.optim_experiment[0].get_summary_statistics(evaluator_selection=["hausdorff", "complexity", "complexity_diff", "geoflow_output"])

{'hausdorff_22_rms': 18.431368109749204,
 'hausdorff_22_mean': 4.641284798394249,
 'hausdorff_22_median': 2.4238131046295166,
 'hausdorff_22_min': 0.05699934437870979,
 'hausdorff_22_max': 179.67298889160156,
 'rms_min_dist_22_rms': 4.342110122659195,
 'rms_min_dist_22_mean': 0.9740868862145297,
 'rms_min_dist_22_median': 0.3189330809084752,
 'rms_min_dist_22_min': 0.01856285590827789,
 'rms_min_dist_22_max': 42.43379572464923,
 'n_faces_22_count': 99.0,
 'n_faces_22_mean': 179.66666666666666,
 'n_faces_22_std': 204.37490560442183,
 'n_faces_22_min': 12.0,
 'n_faces_22_median': 104.0,
 'n_faces_22_max': 966.0,
 'n_faces_22_diff_mean': 66.37373737373737,
 'n_faces_22_diff_std': 93.45899306577779,
 'n_faces_22_diff_min': -98.0,
 'n_faces_22_diff_median': 34.0,
 'n_faces_22_diff_max': 460.0,
 'n_faces_22_abs_diff_mean': 73.70707070707071,
 'n_faces_22_abs_diff_std': 87.7316470167434,
 'n_faces_22_abs_diff_min': 0.0,
 'n_faces_22_abs_diff_median': 36.0,
 'n_faces_22_abs_diff_max': 460.0,
 

Summary stats from second attempt (after original):

In [14]:
e[48].recon_optim.optim_experiment[0].get_summary_statistics(evaluator_selection=["hausdorff", "complexity", "complexity_diff", "geoflow_output"])

{'hausdorff_22_rms': 3.784218639987789,
 'hausdorff_22_mean': 2.9096434658600225,
 'hausdorff_22_median': 2.4238131046295166,
 'hausdorff_22_min': 0.05699934437870979,
 'hausdorff_22_max': 8.714778900146484,
 'rms_min_dist_22_rms': 0.8293215807428279,
 'rms_min_dist_22_mean': 0.5598914675462001,
 'rms_min_dist_22_median': 0.31443842619118695,
 'rms_min_dist_22_min': 0.018620240628682554,
 'rms_min_dist_22_max': 3.4563919608279825,
 'n_faces_22_count': 99.0,
 'n_faces_22_mean': 179.7171717171717,
 'n_faces_22_std': 204.65456413305475,
 'n_faces_22_min': 12.0,
 'n_faces_22_median': 104.0,
 'n_faces_22_max': 970.0,
 'n_faces_22_diff_mean': 66.42424242424242,
 'n_faces_22_diff_std': 93.57961697095381,
 'n_faces_22_diff_min': -98.0,
 'n_faces_22_diff_median': 34.0,
 'n_faces_22_diff_max': 460.0,
 'n_faces_22_abs_diff_mean': 73.6969696969697,
 'n_faces_22_abs_diff_std': 87.9072394392271,
 'n_faces_22_abs_diff_min': 0.0,
 'n_faces_22_abs_diff_median': 36.0,
 'n_faces_22_abs_diff_max': 460.0,


Summary stats from third attempt (after original):

In [21]:
e[48].recon_optim.optim_experiment[0].get_summary_statistics(evaluator_selection=["hausdorff", "complexity", "complexity_diff", "geoflow_output"])

{'hausdorff_22_rms': 3.7839560054219636,
 'hausdorff_22_mean': 2.909453480194012,
 'hausdorff_22_median': 2.4238131046295166,
 'hausdorff_22_min': 0.05699934437870979,
 'hausdorff_22_max': 8.714778900146484,
 'rms_min_dist_22_rms': 0.8287265633495821,
 'rms_min_dist_22_mean': 0.5598618080936618,
 'rms_min_dist_22_median': 0.31675773896925224,
 'rms_min_dist_22_min': 0.018545360709879277,
 'rms_min_dist_22_max': 3.4506744003483942,
 'n_faces_22_count': 99.0,
 'n_faces_22_mean': 179.8989898989899,
 'n_faces_22_std': 205.41796589698527,
 'n_faces_22_min': 12.0,
 'n_faces_22_median': 104.0,
 'n_faces_22_max': 994.0,
 'n_faces_22_diff_mean': 66.60606060606061,
 'n_faces_22_diff_std': 94.07149020301898,
 'n_faces_22_diff_min': -98.0,
 'n_faces_22_diff_median': 34.0,
 'n_faces_22_diff_max': 460.0,
 'n_faces_22_abs_diff_mean': 73.91919191919192,
 'n_faces_22_abs_diff_std': 88.38144102529496,
 'n_faces_22_abs_diff_min': 0.0,
 'n_faces_22_abs_diff_median': 36.0,
 'n_faces_22_abs_diff_max': 460.0

Summary stats for first attempt (after original) of scenario 37's optim_0000:

In [37]:
e[scenario_id].recon_optim.optim_experiment[0].get_summary_statistics(evaluator_selection=["hausdorff", "complexity", "complexity_diff", "geoflow_output"])

{'hausdorff_22_rms': 4.49230886188208,
 'hausdorff_22_mean': 3.3805000903178946,
 'hausdorff_22_median': 2.4176244735717773,
 'hausdorff_22_min': 0.06685002148151398,
 'hausdorff_22_max': 10.628173828125,
 'rms_min_dist_22_rms': 0.9740300705975379,
 'rms_min_dist_22_mean': 0.6690254074447283,
 'rms_min_dist_22_median': 0.36063238556164406,
 'rms_min_dist_22_min': 0.019713106064374788,
 'rms_min_dist_22_max': 2.830117127429148,
 'n_faces_22_count': 99.0,
 'n_faces_22_mean': 140.66666666666666,
 'n_faces_22_std': 145.04770503077214,
 'n_faces_22_min': 12.0,
 'n_faces_22_median': 82.0,
 'n_faces_22_max': 874.0,
 'n_faces_22_diff_mean': 27.373737373737374,
 'n_faces_22_diff_std': 61.319435587646,
 'n_faces_22_diff_min': -250.0,
 'n_faces_22_diff_median': 28.0,
 'n_faces_22_diff_max': 166.0,
 'n_faces_22_abs_diff_mean': 49.41414141414141,
 'n_faces_22_abs_diff_std': 45.280984453541194,
 'n_faces_22_abs_diff_min': 0.0,
 'n_faces_22_abs_diff_median': 36.0,
 'n_faces_22_abs_diff_max': 250.0,
 

In [38]:
e[scenario_id].recon_optim.optim_experiment[0].concat_evaluation_results(evaluator_selection=["hausdorff", "complexity", "complexity_diff", "geoflow_output"])

In [42]:
e[scenario_id].recon_optim.optim_experiment.run_steps(Scenario.setup_evaluation, lods=["2.2"])
e[scenario_id].recon_optim.optim_experiment.compute_summary_statistics(evaluator_selection=["hausdorff", "complexity", "complexity_diff", "geoflow_output"], ignore_missing=True)


Running 'setup_evaluation' for optim_0000 ...

Finished 'setup_evaluation' for optim_0000 after 0:00:00.018004.

Running 'setup_evaluation' for optim_0001 ...

Finished 'setup_evaluation' for optim_0001 after 0:00:00.018995.

Running 'setup_evaluation' for optim_0002 ...

Finished 'setup_evaluation' for optim_0002 after 0:00:00.015002.

Running 'setup_evaluation' for optim_0003 ...

Finished 'setup_evaluation' for optim_0003 after 0:00:00.016003.

Running 'setup_evaluation' for optim_0004 ...

Finished 'setup_evaluation' for optim_0004 after 0:00:00.015249.

Running 'setup_evaluation' for optim_0005 ...

Finished 'setup_evaluation' for optim_0005 after 0:00:00.016001.

Running 'setup_evaluation' for optim_0006 ...

Finished 'setup_evaluation' for optim_0006 after 0:00:00.017971.

Running 'setup_evaluation' for optim_0007 ...

Finished 'setup_evaluation' for optim_0007 after 0:00:00.016993.

Running 'setup_evaluation' for optim_0008 ...

Finished 'setup_evaluation' for optim_0008 after

Dataset-level check for outliers in the OBJ files does not detect these faulty vertices in individual buildings. Should have done a building-level check!

In [22]:
from experiment.obj_file import OBJFile

In [39]:
o = OBJFile(r"C:\Users\Florian\Data\city-to-scan-to-city\utrecht_10-492-594_v2\06_reconstruction_optimization\scenario_048\07_reconstruction\optim_0000_backup\output\model_lod22.obj")

In [40]:
o.has_outlier_vertices(threshold=1e3)

False